# Wordsearch
Given a text file, consisting of three parts (a grid size, a grid, and a list of words), find:
* the words present in the grid, 
* the longest word present in the grid, 
* the number of words not present in the grid, 
* the longest word not present that can be formed from the leftover letters

The only words that need be considered are the ones given in the list in the puzzle input.

The puzzle consists of:
1. A line consisting of _w_`x`_h_, where _w_ and _h_ are integers giving the width and height of the grid.
2. The grid itself, consisting of _h_ lines each of _w_ letters.
3. A list of words, one word per line, of arbitrary length. 

## Example


￼`...p.mown.
.sdse..ee.
.e.elad.cr
pi.dtir.ah
rzsiwovspu
oawh.kieab
brow.c.rda
ecnotops.r
d.kc.d...b
.staple...`

```
fhjpamownq
wsdseuqeev
ieaeladhcr
piedtiriah
rzsiwovspu
oawhakieab
browpcfrda
ecnotopssr
dikchdnpnb
bstapleokr
```

14 words added;  6 directions

Present: apace cowhides crazies dock knows lived mown pears probed rhubarb rioted staple tops wide

Decoys: adapting bombing boor brick cackles carnal casino chaplets chump coaster coccyxes coddle collies creels crumbled cunt curds curled curlier deepen demeanor dicier dowses ensuing faddish fest fickler foaming gambol garoting gliding gristle grunts guts ibex impugns instants kielbasy lanyard loamier lugs market meanly minuend misprint mitts molested moonshot mucking oaks olives orgasmic pastrami perfect proceed puckered quashed refined regards retraces revel ridges ringlet scoff shinier siren solaria sprain sunder sunup tamped tapes thirds throw tiller times trains tranquil transfix typesets uric wariness welts whimsy winced winced

Decoys: fickler, adapting, chump, foaming, molested, carnal, crumbled, guts, minuend, bombing, winced, coccyxes, solaria, shinier, cackles

All words: adapting, apace, bombing, cackles, carnal, chump, coccyxes, cowhides, crazies, crumbled, dock, fickler, foaming, guts, knows, lived, minuend, molested, mown, pears, probed, rhubarb, rioted, shinier, solaria, staple, tops, wide, winced

Directions:  [('probed', '`(True, 3, 0, <Direction.down: 4>)`'), ('staple', '`(True, 9, 1, <Direction.right: 2>)`'), ('rioted', '`(True, 6, 7, <Direction.upleft: 5>)`'), ('cowhides', '`(True, 8, 3, <Direction.up: 3>)`'), ('tops', '`(True, 7, 4, <Direction.right: 2>)`'), ('knows', '`(True, 8, 2, <Direction.up: 3>)`'), ('lived', '`(True, 2, 4, <Direction.downright: 8>)`'), ('rhubarb', '`(True, 2, 9, <Direction.down: 4>)`'), ('crazies', '`(True, 7, 1, <Direction.up: 3>)`'), ('dock', '`(True, 8, 5, <Direction.up: 3>)`'), ('apace', '`(True, 5, 8, <Direction.up: 3>)`'), ('mown', '`(True, 0, 5, <Direction.right: 2>)`'), ('pears', '`(True, 0, 3, <Direction.downright: 8>)`'), ('wide', '`(True, 4, 4, <Direction.upright: 6>)`')]

In [1]:
import string
import re
import collections
import copy
import os

from enum import Enum
Direction = Enum('Direction', 'left right up down upleft upright downleft downright')
    
delta = {Direction.left: (0, -1),Direction.right: (0, 1), 
         Direction.up: (-1, 0), Direction.down: (1, 0), 
         Direction.upleft: (-1, -1), Direction.upright: (-1, 1), 
         Direction.downleft: (1, -1), Direction.downright: (1, 1)}

cat = ''.join
wcat = ' '.join
lcat = '\n'.join

In [2]:
def empty_grid(w, h):
    return [['.' for c in range(w)] for r in range(h)]

In [3]:
def show_grid(grid):
    return lcat(cat(r) for r in grid)

In [4]:
def indices(grid, r, c, l, d):
    dr, dc = delta[d]
    w = len(grid[0])
    h = len(grid)
    inds = [(r + i * dr, c + i * dc) for i in range(l)]
    return [(i, j) for i, j in inds
           if i >= 0
           if j >= 0
           if i < h
           if j < w]

In [5]:
def gslice(grid, r, c, l, d):
    return [grid[i][j] for i, j in indices(grid, r, c, l, d)]

In [6]:
def set_grid(grid, r, c, d, word):
    for (i, j), l in zip(indices(grid, r, c, len(word), d), word):
        grid[i][j] = l
    return grid

In [7]:
def present_many(grid, words):
    w = len(grid[0])
    h = len(grid)
    wordlens = set(len(w) for w in words)
    presences = []
    for r in range(h):
        for c in range(w):
            for d in Direction:
                for wordlen in wordlens:
                    word = cat(gslice(grid, r, c, wordlen, d))
                    if word in words:
                        presences += [(word, r, c, d)]
    return set(presences)

In [30]:
def read_wordsearch(fn):
    lines = [l.strip() for l in open(fn).readlines()]
    w, h = [int(s) for s in lines[0].split('x')]
    grid = lines[1:h+1]
    words = set(lines[h+1:])
    return w, h, grid, words

# All wordsearch puzzles

In [9]:
def read_all_wordsearch(fn):
    with open(fn) as f:
        text = f.read().strip()
        puzzles_text = text.split('\n\n')
        puzzles = []
        for p in puzzles_text:
            lines = p.splitlines()
            w, h = [int(s) for s in lines[0].split('x')]
            grid = lines[1:h+1]
            words = lines[h+1:]
            puzzles += [(w, h, grid, words)]
    return puzzles

## Huge wordsearch

In [31]:
puzzle = read_wordsearch('10-wordsearch.txt')
puzzle[:2]

(100, 100)

## Part 1

In [32]:
def found_words_length(puzzle):
    width, height, grid, words = puzzle
    return sum(len(p[0]) for p in present_many(grid, words))

def total_found_words_length(puzzles):
    return sum(found_words_length(p) for p in puzzles)

In [33]:
found_words_length(puzzle)

8092

In [34]:
%%timeit
found_words_length(puzzle)

1 loop, best of 3: 6.79 s per loop


In [35]:
width, height, grid, words = puzzle
presences = present_many(grid, words)
found_words = [p[0] for p in presences]
len(presences), len(found_words), len(set(found_words))

(1149, 1149, 1149)

In [38]:
%%timeit
presences = present_many(grid, words)

1 loop, best of 3: 6.76 s per loop


In [37]:
found_words = [p[0] for p in presences]
len(found_words), len(set(found_words))

(1149, 1149)

## Part 2

In [15]:
def max_unfound_word_length(puzzle):
    width, height, grid, words = puzzle
    presences = present_many(grid, words)
    used_words = [p[0] for p in presences]
    unused_words = [w for w in words if w not in used_words]
    
    unused_grid = [[c for c in r] for r in grid]
    for w, r, c, d in presences:
        set_grid(unused_grid, r, c, d, '.' * len(w))
    unused_letters = [c for l in unused_grid for c in l if c != '.']
    unused_letter_count = collections.Counter(unused_letters)
    
    makeable_words = []
    for w in unused_words:
        unused_word_count = collections.Counter(w)
        if all(unused_word_count[l] <= unused_letter_count[l] for l in unused_word_count):
            makeable_words += [w]
    lwm = max(len(w) for w in makeable_words)
    return lwm

In [16]:
def unused_letters(puzzle):
    width, height, grid, words = puzzle
    presences = present_many(grid, words)
    used_words = [p[0] for p in presences]
    unused_words = [w for w in words if w not in used_words]
    
    unused_grid = [[c for c in r] for r in grid]
    for w, r, c, d in presences:
        set_grid(unused_grid, r, c, d, '.' * len(w))
    unused_letters = [c for l in unused_grid for c in l if c != '.']
    unused_letter_count = collections.Counter(unused_letters)
    
    return used_words, unused_letter_count

In [17]:
def unused_vowels(puzzle):
    width, height, grid, words = puzzle
    presences = present_many(grid, words)
    used_words = [p[0] for p in presences]
    unused_words = [w for w in words if w not in used_words]
    
    unused_grid = [[c for c in r] for r in grid]
    for w, r, c, d in presences:
        set_grid(unused_grid, r, c, d, '.' * len(w))
    unused_vowel_count = sum(1 for l in unused_grid for c in l if c in 'aeiou')
    return unused_vowel_count

In [18]:
def total_max_unfound_word_length(puzzles):
    return sum(max_unfound_word_length(p) for p in puzzles)

In [19]:
unused_vowels(puzzle)

594

In [20]:
%%timeit
unused_vowels(puzzle)

1 loop, best of 3: 30.8 s per loop


In [21]:
# max_unfound_word_length(puzzle)

In [22]:
# %%timeit
# max_unfound_word_length(puzzle)

In [23]:
uw, unlc = unused_letters(puzzle)
sum(unlc[l] for l in unlc)

2217